In [14]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Batch, Data, DataLoader, InMemoryDataset
from torch_geometric.nn import MessagePassing
import torch_geometric.transforms as T

import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm import tqdm, trange
from IPython.display import clear_output
from typing import Literal

torch.cuda.is_available()

True

In [15]:
class RelPosConv(MessagePassing):
    def __init__(self, emb_dim, msg_dim, out_channels, aggr = 'mean'):
        super().__init__(aggr=aggr)
        self.edge_mlp = torch.nn.Linear(emb_dim+emb_dim,msg_dim)
        self.update_mlp = torch.nn.Linear(emb_dim+msg_dim,out_channels)
        self.reset_parameters()

    def forward(self,x,edge_attr,edge_index):
        out = self.propagate(edge_index,x=x,edge_attr=edge_attr)
        return out
    
    def message(self, x_j, edge_attr):
        tmp = torch.cat([x_j, edge_attr],dim=1)
        return self.edge_mlp(tmp)
    
    def update(self,aggr_out,x):
        cat = torch.cat([x, aggr_out],dim=1)
        return self.update_mlp(cat)

In [16]:
class GCONV_Model_RelPos(torch.nn.Module):
    def __init__(self, emb_dim=64, msg_dim=64, node_dim=7, edge_dim=4, out_dim = 3):
        super(GCONV_Model_RelPos,self).__init__()
        self.node_embed = torch.nn.Linear(node_dim,emb_dim)
        self.edge_embed = torch.nn.Linear(edge_dim,emb_dim)
        self.conv1 = RelPosConv(emb_dim,msg_dim,emb_dim)
        self.conv2 = RelPosConv(emb_dim,msg_dim,emb_dim)
        self.conv3 = RelPosConv(emb_dim,msg_dim,out_dim)
        self.mode = "delta"

    def forward(self,data):
        x, edge_attr, edge_index = data.x, data.edge_attr, data.edge_index
        x = self.node_embed(x)
        edge_attr = self.edge_embed(edge_attr)
        x = F.relu(x)
        x = self.conv1(x, edge_attr, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_attr, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_attr, edge_index)
        return x

In [17]:
class Trainer:
    def __init__(self,model,dataset_train,dataset_val,batch_size,lr,epochs,model_name,loss_fn=torch.nn.MSELoss()):
        self.model = model
        self.batch_size = batch_size
        self.lr = lr
        self.epochs = epochs
        self.loss_fn = loss_fn
        self.model_name = model_name

        self.device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')
        print("Device: ", self.device)
        self.model.to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=self.lr)
        self.train_dl = self.make_data_loader(dataset_train, shuffle=True)
        self.val_dl = self.make_data_loader(dataset_val, shuffle=False)

    def make_data_loader(self, dataset, shuffle):
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)
    
    def loss_batch(self, batch,opt=None):
        out = self.model(batch)
        mask = np.concatenate(batch.mask)
        loss =self.loss_fn(out[mask], batch.y)

        if opt is not None:
            loss.backward()
            opt.step()
            opt.zero_grad()
        return loss.item()

    def batch_loop(self, dataloader, loss_list, axes, opt=None):
        mean_loss = 0
        for i, batch in enumerate(dataloader):
            batch_loss = self.loss_batch(batch.to(self.device), opt)
            mean_loss += batch_loss
        mean_loss /= i
        loss_list.append(mean_loss)
        axes[0].plot(loss_list)
        axes[1].plot(loss_list[-5:])
        return mean_loss,loss_list

    def train_loop(self):
        train_loss, val_loss = [], []
        best_model_loss = np.inf
        for epoch in tqdm(range(self.epochs)):
            clear_output(wait=True)
            fig, axes = plt.subplots(1,2)
            fig.set_figwidth(15)

            self.model.train()  
            mean_train_loss, train_loss = self.batch_loop(self.train_dl,train_loss,axes,self.optimizer)

            self.model.eval()
            with torch.inference_mode():
                mean_val_loss, val_loss = self.batch_loop(self.val_dl,val_loss,axes)

            for ax in axes: ax.set(xlabel='Epoch',ylabel='Loss'), ax.set_ylim(ymin=0), ax.set_xlim(xmin=0)

            plt.show()

            if mean_val_loss < best_model_loss:
                best_model_loss = mean_val_loss
                torch.save(self.model.state_dict(),os.path.join(os.getcwd(),"Models",self.model_name))

            print(f"Epoch {epoch}, Mean Train Loss: {mean_train_loss}, Mean Validation Loss: {mean_val_loss}")
        np.save(f"{os.getcwd()}\\Models\\{self.model_name}_Training_Loss",train_loss)
        np.save(f"{os.getcwd()}\\Models\\{self.model_name}_Validation_Loss",val_loss)

In [18]:
def GetModel(dataset_name,model_name,edge_dim):
    model = GCONV_Model_RelPos(edge_dim=edge_dim)
    try: model.load_state_dict(torch.load(f"{os.getcwd()}\\Models\\{dataset_name}_GCONV_Model_{model_name}"))
    except: print("No Trained model")
    return model

In [19]:
def GetScales(dataset):
    scale_pos = 1.0/dataset.pos.abs().max()
    scale_x = 1/dataset.x.max(dim=0,keepdim=False)[0]
    return scale_pos,scale_x

def NormalizePos(dataset,scale_pos=None,scalex=None):
    if scale_pos is None:
        scale_pos = 1.0/dataset.pos.abs().max()
    
    if scalex is None:
        scalex = 1/dataset.x.max(dim=0,keepdim=False)[0]

    assert dataset.pos is not None
    dataset.pos *= scale_pos
    dataset.x *= scalex
    return dataset, scale_pos, scalex

In [24]:
from Encoding import ConstructTopology, ToPytorchData
class DEM_Dataset(InMemoryDataset):
    def __init__(self,file_name: str,data_split,
                 Dataset_type: Literal["train","validate","test"],
                 mode: Literal["cart","delta"],
                 force_reload=False,pre_transform=None, transform=None, pre_filter=None,
                 root: str = os.getcwd()+r"\Data"):
        
        self.raw_data_path = f"{root}\\raw"
        self.processed_data_path = f"{root}\\processed"
        self.file_name = file_name
        self.Dataset_type = Dataset_type
        self.mode = mode
        self.data_split = data_split
        super().__init__(root, transform, pre_transform,pre_filter,force_reload=force_reload)
        self.load(os.path.join(self.processed_data_path,self.processed_file_names[0]))

    @property 
    def raw_file_names(self):
        return[f"{self.file_name}_Data.npy",
               f"{self.file_name}_Topology.npy",
               f"{self.file_name}_BC.npy"]
    
    @property
    def processed_file_names(self):
        if self.pre_filter is None: return [f"{self.file_name}_{self.mode}_{self.Dataset_type}.pt"]
        else: return [f"{self.file_name}_{self.mode}_{self.Dataset_type}_init.pt"]
    
    def download(self):
        pass
    
    # Load data and split them according to dataset split
    def LoadSimTop(self,i):
        data = np.load(f"{self.raw_data_path}\\{self.raw_file_names[i]}",allow_pickle=True)
        Daset_type_idx = {"train":0,"validate":1,"test":2}[self.Dataset_type]
        splits=np.array(self.data_split)*data.shape[0]
        return np.split(data,splits.astype(int))[Daset_type_idx]

    def process(self):
        data_list = []
        data_agr,top_agr,bc= [self.LoadSimTop(i) for i in [0,1,2]]
            
        if self.pre_filter is not None:
            simulations = self.pre_filter(simulations)

        for sim, bc in tqdm(zip(data_agr,bc)):
            R_avg = sim[0][:,3].mean()
            topology = ConstructTopology(sim[0],bc,6*R_avg)-1
            for t in np.arange(len(sim)-1):
                par_data = sim[t]
                label_data = sim[t+1]
                data = ToPytorchData(par_data,bc,None,topology,label_data)[0]
                data_list.append(data)

        if self.Dataset_type == "train":
            scale_pos, scale_x = GetScales(Batch.from_data_list(data_list))
            torch.save(scale_pos,os.path.join(self.processed_data_path,f"{self.file_name}_scale_pos.pt"))
            torch.save(scale_x,os.path.join(self.processed_data_path,f"{self.file_name}_scale_x.pt"))

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]   
                
        self.save(data_list, os.path.join(self.processed_data_path,self.processed_file_names[0]))

In [21]:
from torch_geometric.transforms import BaseTransform
class NormalizePos(BaseTransform):
    r"""Centers and normalizes node positions to the interval :math:`(-1, 1)`
    (functional name: :obj:`normalize_scale`).
    """
    def __init__(self,file_name):
        self.path_scale_pos = os.path.join(os.getcwd(),"Data","processed",f"{file_name}_scale_pos.pt")
        self.path_scale_x   = os.path.join(os.getcwd(),"Data","processed",f"{file_name}_scale_x.pt")
    def forward(self, data: Data) -> Data:
        scale_pos = torch.load(self.path_scale_pos)
        scale_x   = torch.load(self.path_scale_x)
        data.pos *= scale_pos
        data.x   *= scale_x

        return data

In [25]:
file_name="N400"
model_ident = "Norm_01"
data_split=[0.85, 0.95]
pre_transform = T.Compose([NormalizePos(file_name),
                           T.Cartesian(False),
                           T.Distance(norm=False,cat=True)])
transform       = None
force_reload    = True
train           = True

if train == True:
    dataset_train     = DEM_Dataset(file_name, data_split,"train"   ,'delta', force_reload, pre_transform,transform)
    dataset_val       = DEM_Dataset(file_name, data_split,"validate",'delta', force_reload, pre_transform,transform)
    dataset_test      = DEM_Dataset(file_name, data_split,"test"    ,'delta', force_reload, pre_transform,transform)

    model = GetModel("N400",model_ident,edge_dim=4)
    trainer = Trainer(model, dataset_train,dataset_val,
                      batch_size=64,
                      lr=0.0000001,
                      epochs=2000,
                      model_name=f"{file_name}_GCONV_Model_{model_ident}")
    trainer.train_loop()

Processing...
289it [03:47,  1.27it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 204320 bytes.

In [ ]:
print(dataset_train.pos.min(),dataset_train.pos.max())
print(dataset_val.pos.min(),dataset_val.pos.max())
print(dataset_test.pos.min(),dataset_test.pos.max())

print(dataset_train.x.max(dim=0)[0])
print(dataset_val.x.max(dim=0)[0])
print(dataset_test.x.max(dim=0)[0])


In [ ]:
def FilterStart(dataset):
    idx = np.nonzero([data.time == 0 for data in dataset])
    return torch.utils.data.Subset(dataset_test,idx)[0]

#dataset_test_start = FilterStart(dataset_test)

In [ ]:
from Encoding import ToPytorchData, GetLength

def GetLimits(data):
    max = [torch.max(data.x[:,i]) for i in [0, 1, 2]]
    min = [torch.min(data.x[:,i]) for i in [0, 1, 2]]
    max = torch.stack(max)
    min = torch.stack(min)
    limits = torch.stack([min,max],dim=1)
    return limits


In [ ]:
class LearnedSimulator:
    def BCrollout(self):
        print("Calculating BC")
        BC_rollout = np.empty((GetLength(self.par_data),6,9))
        BC_t=np.copy(self.BC)
        for t in trange(GetLength(self.par_data)):
            BC_t[:,:3] = self.BC[:,:3]+(t+1)*self.BC[:,-3:] 
            BC_rollout[t] = BC_t
        return BC_rollout
    
    def GroundTruth_Rollout(self):
        print("Collecting Ground Truth Rollout")
        GroundTruth = np.empty(GetLength(self.par_data),dtype=object)
        for t in trange(GetLength(self.par_data)):
            data = ToPytorchData(self.par_data[t],self.BC_rollout[t],self.tol,self.topology[t])[0]
            GroundTruth[t] = data
        return GroundTruth
    
    def MLRollout(self):
        print("Calculating Learned Rollout")
        with torch.inference_mode():
            
            ML_Rollout = np.empty((self.timesteps),dtype=object)
            par_inp = self.par_data[0]
            ML_Rollout[0], MatlabTopology = ToPytorchData(par_inp,self.BC_rollout[0],self.tol)

            for t in tqdm(range(1,self.timesteps)):
                input_data = ToPytorchData(par_inp,self.BC_rollout[t],self.tol,MatlabTopology)[0]
                if self.model.mode == 'delta': 
                    #transform = T.Compose([T.Cartesian(False), T.Distance(norm=False,cat=True)])
                    transform = T.Cartesian(False)
                input_data.to(self.device)

                output = self.model(transform(input_data))
                par_inp[:,:3] = par_inp[:,:3]+output[input_data.mask].cpu().numpy()
                output_data = ToPytorchData(par_inp,self.BC_rollout[t],self.tol,MatlabTopology)
                ML_Rollout[t] = output_data[0]
        return ML_Rollout
    
    def __init__(self,model,data_agr, top_agr,BC_agr,i: int,tol,timesteps=100):
        self.device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')
        self.BC = BC_agr[i]
        self.par_data = data_agr[i]
        self.topology = top_agr[i]
        self.model = model.to(self.device)
        self.data_list = []
        self.timesteps = timesteps
        self.tol = tol
        self.BC_rollout = self.BCrollout()
        self.GroundTruth = self.GroundTruth_Rollout()
        self.ML_rollout = self.MLRollout()

In [ ]:
from Encoding import load
def SplitData(dataset_name,data_split):
    loaded_data = load(dataset_name)
    splits=np.array(data_split)*loaded_data[0].shape[0]
    test_data = [np.split(data,splits.astype(int))[2] for data in loaded_data]
    return test_data

In [ ]:
dataset_name = "2Sphere"
model_name = "RelPos_02"
tol = 0.5
from Plotting import PlotXYZ
for sample_idx in range (30):
    model = GetModel(dataset_name,model_name,edge_dim=3)
    AggregatedArgs = SplitData(dataset_name,data_split)
    Rollout = LearnedSimulator(model,*AggregatedArgs,i=sample_idx,tol=tol,timesteps=100)

    if dataset_name == "2Sphere": 
        PlotXYZ(Rollout,t_max=100)

In [ ]:
from Plotting import GetInternalStressRollout
stress = GetInternalStressRollout(Rollout)
torch.set_printoptions(4)
print("Stress at time 0")
print(torch.round(stress[0],decimals=8)),
print("\nStress at time 99")
print(torch.round(stress[-1],decimals=1))

In [ ]:
from Plotting import PlotContactVectorAndForce, GetAllContactpoints,AxesLimits
data = Rollout.GroundTruth[0]
BC = Rollout.BC_rollout[3]
fig,axs = PlotContactVectorAndForce(data,BC)
for ax in axs: AxesLimits(ax,BC)

In [ ]:
from Plotting import PlotGraphComparison
save = False
show = False
for t in range(20,Rollout.timesteps,20):  
    fig = PlotGraphComparison(t,Rollout,sample_idx,tol,plot_lines=True)
    if save == True: plt.savefig(f"{os.getcwd()}\\Figures\\Plots\\Graph_Sample{sample_idx}_Time{t}_Tol{str(tol)[2:]}.png",bbox_inches='tight')     
    if show == True: plt.show()
fig = PlotGraphComparison(Rollout.timesteps-1,Rollout,sample_idx,tol,plot_lines=False) 
if save == True: plt.savefig(f"{os.getcwd()}\\Figures\\Plots\\Graph_Sample{sample_idx}_Time{t}_Tol{str(tol)[2:]}.png",bbox_inches='tight')  
if show == True: plt.show()

In [ ]:
import pyvista as pv
from Plotting import MakeGIF, PlotMeshNormals


#datalist = Rollout.ML_rollout
#gifname = f"ML_2_{sample_idx}_Tol{str(tol)[2:]}"
gifname = "Test"
datalist = Rollout.GroundTruth
gifname = f"Groundtruth_2_{sample_idx}_Tol{str(tol)[2:]}"
MakeGIF(datalist,gifname,fps=8,color='lightblue',deformation=True)

#data = Rollout.ML_rollout[10]
#data = Rollout.GroundTruth[0]
#PlotMeshNormals(data)


In [ ]:
data = Rollout.ML_rollout[0]
#data = Rollout.GroundTruth[0]